In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("akashshingha850/mrl-eye-dataset")

print("Path to dataset files:", path)

100%|██████████| 329M/329M [00:02<00:00, 160MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/akashshingha850/mrl-eye-dataset/versions/4


In [2]:
import os
os.listdir("/root/.cache/kagglehub/datasets/akashshingha850/mrl-eye-dataset/versions/4/data/val")

['sleepy', 'awake']

In [3]:
train_folder_path='/root/.cache/kagglehub/datasets/akashshingha850/mrl-eye-dataset/versions/4/data/train'
test_folder_path='/root/.cache/kagglehub/datasets/akashshingha850/mrl-eye-dataset/versions/4/data/test'
val_folder_path="/root/.cache/kagglehub/datasets/akashshingha850/mrl-eye-dataset/versions/4/data/val"
# Device setup

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
print(f"TensorFlow version: {tf.__version__}")

# GPU Configuration for Google Colab
def configure_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU configured successfully! Available GPUs: {len(gpus)}")
            print(f"GPU details: {gpus[0]}")
            return True
        except RuntimeError as e:
            print(f"❌ GPU configuration failed: {e}")
            return False
    else:
        print("ℹ️ No GPU available, using CPU")
        return False
gpu_available = configure_gpu()

TensorFlow version: 2.18.0
✅ GPU configured successfully! Available GPUs: 1
GPU details: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [11]:

IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 5

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_folder_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset='training',
    label_mode='categorical',
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_folder_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset='validation',
    label_mode='categorical',
    seed=42
)


Found 50937 files belonging to 2 classes.
Using 40750 files for training.
Found 16980 files belonging to 2 classes.
Using 3396 files for validation.


In [15]:
base_model = tf.keras.applications.ResNet50(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  #freeze base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(2, activation='softmax')  # classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1
)

Epoch 1/5
2547/2547 ━━━━━━━━━━━━━━━━━━━━ 140s 50ms/step - accuracy: 0.9445 - loss: 0.1492 - val_accuracy: 0.9753 - val_loss: 0.0677
Epoch 2/5
2547/2547 ━━━━━━━━━━━━━━━━━━━━ 118s 46ms/step - accuracy: 0.9669 - loss: 0.0893 - val_accuracy: 0.9756 - val_loss: 0.0598
Epoch 3/5
2547/2547 ━━━━━━━━━━━━━━━━━━━━ 143s 47ms/step - accuracy: 0.9712 - loss: 0.0774 - val_accuracy: 0.9761 - val_loss: 0.0590
Epoch 4/5
2547/2547 ━━━━━━━━━━━━━━━━━━━━ 144s 47ms/step - accuracy: 0.9744 - loss: 0.0686 - val_accuracy: 0.9806 - val_loss: 0.0500
Epoch 5/5
2547/2547 ━━━━━━━━━━━━━━━━━━━━ 139s 46ms/step - accuracy: 0.9776 - loss: 0.0600 - val_accuracy: 0.9850 - val_loss: 0.0415


In [17]:
os.mkdir('/content/saved_model')

In [19]:
model.save('/content/saved_model/resnet.keras')